<a href="https://colab.research.google.com/github/RodrigodeFeo/8DTS/blob/main/Trabalho_Integrado_CV_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python-headless
!pip install mediapipe
!pip install dlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [2]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2024-07-28 20:16:24--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M   130MB/s    in 0.5s    

2024-07-28 20:16:24 (130 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
from IPython.display import display, Javascript, HTML

HTML('''
<video autoplay playsinline id="video" width="640" height="480"></video>
<button id="captureButton">Capturar Frame</button>
<button id="stopButton">Parar Webcam</button>
<button id="startButton">Iniciar Webcam</button>
<script>
  const video = document.getElementById('video');
  const button = document.getElementById('captureButton');
  let stream;

  async function setupCamera() {
    stream = await navigator.mediaDevices.getUserMedia({ video: true });
    video.srcObject = stream;
    await new Promise((resolve) => (video.onloadedmetadata = resolve));
    video.play();
  }

  async function stopCamera() {
    if (stream) {
      stream.getTracks().forEach((track) => track.stop());
      stream.srcObject = null;
      stream = null;
    }
  }

  async function startCamera() {
    await stopCamera();
    await setupCamera();
  }


  function captureFrame() {
    const canvas = document.createElement('canvas');
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    const context = canvas.getContext('2d');
    context.drawImage(video, 0, 0, canvas.width, canvas.height);
    const data = canvas.toDataURL('image/jpeg');
    google.colab.kernel.invokeFunction('notebook.get_frame', [data], {});
    setTimeout(captureFrame, 100);  // Capture a frame every 100ms
  }

  button.onclick = async function() {
    const canvas = document.createElement('canvas');
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    const context = canvas.getContext('2d');
    context.drawImage(video, 0, 0, canvas.width, canvas.height);
    const data = canvas.toDataURL('image/jpeg');
    google.colab.kernel.invokeFunction('notebook.get_frame', [data], {});
  }

  stopButton.onclick = async function() {
    await stopCamera();
  }

  startButton.onclick = async function() {
    await startCamera();
  }

  //setupCamera().then(() => {
  //  captureFrame();
  //});

</script>
''')

Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Imagem salva em /content/captured_frame.jpg
Piscada Não Detectada!
I

In [5]:
import cv2
import numpy as np
import mediapipe as mp
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import dlib

# Inicializa a detecção facial com mediapipe
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# Carrega o detector de marcos faciais de dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Função para calcular a razão de aspecto do olho (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Limite para detectar piscadas
EYE_AR_THRESH = 0.21

# Função para processar o frame
def process_frame(data):
    # Decodifica a imagem
    image_data = data.split(",")[1]
    image = np.frombuffer(base64.b64decode(image_data), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    #Salva imagem sem face detect
    cv2.imwrite('/content/captured_frame.jpg', image)
    print('Imagem salva em /content/captured_frame.jpg')

    # Converte a imagem para RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Processa a imagem para detecção facial
    results = face_detection.process(rgb_image)

    # Desenha as detecções faciais na imagem
    if results.detections:
        for detection in results.detections:
            mp_drawing.draw_detection(image, detection)

        # Converte a imagem para escala de cinza
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detecta faces na imagem
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = np.array([[p.x, p.y] for p in shape.parts()])

            leftEye = shape[42:48]
            rightEye = shape[36:42]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR + rightEAR) / 2.0

            # Desenha os olhos na imagem
            for (x, y) in leftEye:
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            for (x, y) in rightEye:
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

            # Verifica se houve piscada
            if ear < EYE_AR_THRESH:
                cv2.putText(image, "Piscada Detectada!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                print("Piscada Detectada!")
            else:
                cv2.putText(image, "Piscada Nao Detectada!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                print("Piscada Não Detectada!")

    # Limpa a saída anterior
    clear_output(wait=True)

    # Exibe a imagem processada
    cv2_imshow(image)

    # Salva a imagem no ambiente do Colab
    cv2.imwrite('/content/captured_frame_det.jpg', image)
    print('Imagem salva em /content/captured_frame_det.jpg')

# Registro da função para receber frames
output.register_callback('notebook.get_frame', process_frame)